## Supplementary code for paper submission: 'Tracing Semantic Variation in Slang'.

This notebook contains the supplementary data pre-processing code for 'Tracing Semantic Variation in Slang'. Since we cannot publically release all entries from Green's Dictionary of Slang (GDoS) due to copyright terms, this note book illustrates how we pre-process raw data obtained from https://greensdictofslang.com/ and turn the data into a format that can be used to reproduce our experimental results.

Here is a list of non-standard Python packages you'll need. All of which can be obtained using *pip install*.

- numpy
- bs4

In [1]:
import bs4
import pickle
import re
import glob
import numpy as np
from tqdm import trange

In [2]:
from util import GSD_Definition, GSD_Word
from process import process_GSD

For illustration, we include the raw html dumps for 3 dictionary entries for the slang word *beast*. Each file is named after its hash tag organized by the original dictionary. The original entries can be found on the following webpages:

https://greensdictofslang.com/entry/23sqfua

https://greensdictofslang.com/entry/xzzdtua

https://greensdictofslang.com/entry/3e7vqxq

We not first crawl our directory for these hash tags:

In [3]:
word_hash = [s[:-5] for s in glob.glob('*.html')]

In [4]:
word_hash

['xzzdtua', '3e7vqxq', '23sqfua']

The following pre-processing function will then take in a list of hash tags and process the respective html files. A pickle file will be generated for each word entry. Note that we do not collapse homonyms (i.e. same word form with multiple word entries) until the actual experiment.

In [5]:
process_GSD(word_hash, input_dir = "", output_dir = "")

100%|██████████| 3/3 [00:00<00:00, 17.84it/s, d_count=20, w_count=3]


This should generate 3 pickle files which we now load for further pre-processing.

In [6]:
data = [pickle.load(open(h+'.pickle', 'rb')) for h in word_hash]

The following code filters the reference entries according to the set of regions that we are interested in (in our case, US and UK). It also tries to automatically extract valid example usage sentences from the reference entries.

In [7]:
regions = ['[US]', '[UK]']
#regions = ['[US]', '[UK]', '[Aus]']

In [8]:
punctuations = '!\'"#$%&()\*\+,-\./:;<=>?@[\\]^_`{|}~'

re_punc = re.compile(r"["+punctuations+r"]+")
re_space = re.compile(r" +")

re_extract_quote = re.compile(r"[1-9/]+:")
re_extract_quote_all = re.compile(r"[1-9/]+:.*$")

def proc_quote_sent(sent):
    return re_extract_quote.sub(' ', re_extract_quote_all.findall(sent)[0]).strip()

def validate_quote_sent(word, sent):
    tokens = [s.lower() for s in re_space.sub(' ', re_punc.sub('', sent)).split(' ')]
    return word.lower() in tokens

data_proc = []

for i in trange(len(data)):
    w = data[i]
    if w.is_abbr():
        continue
    d_list = []
    for d in w.definitions:
        stamps = d.stamps
        region_set = set([s[1] for s in stamps])
        if np.any([r in region_set for r in regions]):
            new_stamps = [s for s in stamps if np.any([r==s[1] in region_set for r in regions])]
            new_def = GSD_Definition(d.def_sent)
            new_def.stamps = new_stamps
            new_def.contexts = {key:value for key, value in d.contexts.items() if key in new_stamps}
            d_list.append(new_def)
    if len(d_list) > 0:
        new_word = GSD_Word(w.word.replace("\\xe2\\x80\\x99", "'").replace("\\xe2\\x80\\x98", "'"), w.pos, w.homonym)
        new_word.definitions = d_list
        data_proc.append(new_word)

100%|██████████| 3/3 [00:00<00:00, 1552.49it/s]


Here's what the data looks after after pre-processing:

In [9]:
_ = [print(d) for d in data_proc]

[WORD]
beast
[POS]
adj.
[DEFINITIONS]
a general intensifier, both positive and negative
1734 - [UK]
1956 - [UK]
white; pertaining to white culture
1999 - [UK] - I dont want no beast food.
excellent
1997 - [US]

[WORD]
beast
[POS]
v.
[DEFINITIONS]
to molest a child
1994 - [UK]
2012 - [UK]
to subject to physical abuse
2005 - [UK]
2018 - [UK]

[WORD]
beast
[POS]
n.
[DEFINITIONS]
an unpopular or unpleasant person
1577 - [UK]
1580 - [UK]
1599 - [UK] - Drunke? hees a beast and he be drunke, theres no man that is a sober man will be drunk, hees a boy and he be drunke.
1611 - [UK] - Out, filthy beast, I loath thy lookes, / And hate thee like a toad.
1619 - [UK]
1642 - [UK]
1673 - [UK]
1678 - [UK]
1703 - [UK]
1727 - [UK]
1731 - [UK] - They call me Old Fool, and drunken old Beast to my Face.
1821 - [UK] - Dull beast! replied Varney.
1837 - [UK]
1862 - [UK]
1873 - [UK] - Look at him! [...] did you ever see such a Beast as he looks?
1896 - [UK] - I never hated anyone that I know of, but I do hate 

We now save the pre-processed data to be used for experiments. See the notebook *Trace.ipynb* in the code package for how this can be used to reproduce results in our paper.

In [10]:
np.save('GSD_sample_data.npy', data_proc)